<a href="https://colab.research.google.com/github/mrlujanero/03-MAIR--Algoritmos-De-Optimizacion--2019/blob/master/AG3/Ramon_Lujan_Crespo_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AG3 - Actividad Guiada 3
# Ramon Lujan Crespo
# URL - https://github.com/mrlujanero/03-MAIR--Algoritmos-De-Optimizacion--2019/tree/master/AG3

In [1]:
import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de./pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp",file)



('swiss42.tsp', <http.client.HTTPMessage at 0x7fe474313400>)

In [3]:
!pip install tsplib95

    100% |████████████████████████████████| 1.6MB 14.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/44/c0/34/6f98693a554301bdb405f8d65d95bbcd3e50180cbfdd98a94e
Successfully built networkx
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2


In [0]:
import tsplib95
import random
from math import e

problem = tsplib95.load_problem(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

def factorial(n):
  if n == 0:
    return 1
  else:
    return n * factorial(n-1)
  
def crear_solucion(Nodos):
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos)-set({0}) - set(solucion)))]
  return solucion

def distancia(a,b,problem):
  return problem.wfunc(a,b)

def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i],solucion[i+1], problem)
  return distancia_total +distancia(solucion[-1], solucion[0], problem)

In [6]:
def busqueda_aleatoria(problem, N):
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  for i in range(N):
    solucion = crear_solucion(Nodos)
    distancia_solucion = distancia_total(solucion, problem)
    if distancia_solucion < mejor_distancia:
      mejor_solucion = solucion
      mejor_distancia = distancia_solucion
  print("La mejor solucion encontrada es ", end="")
  print(solucion)
  print("con una distancia total de ",end="")
  print(mejor_distancia)
  return mejor_solucion

busqueda_aleatoria(problem, 100)

La mejor solucion encontrada es [0, 3, 22, 38, 24, 18, 12, 8, 5, 32, 36, 14, 37, 34, 26, 10, 40, 33, 39, 25, 6, 20, 28, 21, 35, 13, 9, 7, 15, 23, 30, 17, 2, 1, 41, 19, 31, 4, 11, 29, 16, 27]
con una distancia total de 4254


[0,
 1,
 14,
 7,
 4,
 32,
 2,
 20,
 37,
 10,
 25,
 38,
 9,
 33,
 24,
 34,
 30,
 28,
 22,
 27,
 26,
 29,
 6,
 8,
 19,
 5,
 39,
 35,
 16,
 13,
 40,
 21,
 15,
 31,
 18,
 23,
 41,
 17,
 36,
 3,
 11,
 12]

In [0]:
def genera_vecina(solucion):
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion) - 1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

#la mejor distancia inicial es la de la solucion que se esta pasando, por lo que se busca que siempre mejore la solucion actual
def genera_vecina2(solucion):
  mejor_solucion = solucion
  mejor_distancia = distancia_total(solucion, problem)
  for i in range(1,len(solucion) - 1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

#mediante este metodo se generan todas los vecinos posibles mediante permutacion, y se seleccionan como candidatos aquellos vecinos que mejoren la solución puntual, para luego elegir aleatoriamente uno de ellos
def genera_vecinos_elige_aleatorio(solucion):
  candidatos = [solucion]
  distancia_actual = distancia_total(solucion, problem)
  for i in range(1,len(solucion) - 1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= distancia_actual:
        candidatos.append(vecina)
  return random.choice(candidatos)

#mediante este metodo se generan todos losv vecinos posibles de invertir una sublista aleatoria de la solucion, y se escoge la mejor
def genera_vecina_invertir_sublista(solucion):
  mejor_solucion = solucion
  mejor_distancia = distancia_total(solucion,problem)
  for _ in range(len(solucion)):
    firstIndex = random.randint(1, len(solucion)-3)
    secondIndex = random.randint(firstIndex+1, len(solucion)-1)
    vecina = solucion[:firstIndex] + solucion[secondIndex-1:firstIndex-1:-1] + solucion[secondIndex:]
    distancia_vecina = distancia_total(vecina, problem)
    if distancia_vecina <= mejor_distancia:
      mejor_distancia = distancia_vecina
      mejor_solucion = vecina
  return mejor_solucion

In [73]:
def busqueda_local(problem, N):
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  solucion_referencia = crear_solucion(Nodos)
  
  for i in range(N):
    vecina = genera_vecina_invertir_sublista(solucion_referencia)
    distancia_vecina = distancia_total(vecina,problem)
    if distancia_vecina <= mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      
    solucion_referencia = vecina
    
  print("La mejor solucion encontrada es ", end="")
  print(mejor_solucion)
  print("Con una distancia total de ", end="")
  print(mejor_distancia)
  return mejor_solucion

sol = busqueda_local(problem,100)

La mejor solucion encontrada es [0, 1, 3, 6, 26, 5, 19, 13, 18, 12, 11, 25, 10, 4, 2, 27, 28, 29, 30, 32, 31, 7, 14, 16, 15, 37, 17, 36, 35, 20, 33, 34, 38, 22, 39, 9, 8, 41, 23, 21, 24, 40]
Con una distancia total de 1598


In [0]:
def genera_vecina_aleatorio(solucion):
  i = random.choice(range(1, len(solucion)))
  j = random.choice(list(set(range(1,len(solucion)))-{i}))
  vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  return vecina

def probabilidad(T,d):
  r = random.random()
  return r <= (e**(-1*d)/(T*1.0))

def bajar_temperatura(T):
  return T-1

In [0]:
def recocido_simulado(problem, TEMPERATURA):
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  while TEMPERATURA > 0:
    vecina = genera_vecina_aleatorio(solucion_referencia)
    
    distancia_vecina = distancia_total(vecina,problem)
    
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina)):
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    
    TEMPERATURA = bajar_temperatura(TEMPERATURA)
    
  print("La mejor solucion encontrada es ",end="")
  print(mejor_solucion)
  print("con una distancia total de ",end="")
  print(mejor_distancia)
  return mejor_solucion

sol = recocido_simulado(problem,100)

La mejor solucion encontrada es [0, 4, 2, 41, 23, 25, 29, 22, 33, 35, 28, 26, 1, 5, 17, 32, 12, 9, 13, 39, 24, 21, 10, 16, 37, 19, 8, 18, 34, 11, 27, 20, 6, 3, 30, 40, 7, 15, 31, 14, 36, 38]
con una distancia total de 3983


In [0]:
def hormigas(problem, N):
  
  Nodos = list(problem.get_nodes())
  Aristas = list(problem.get_edges())
  
  T = [[1 for _ in range(len(Nodos))] for _ in range(len(Nodos))]
  
  Hormiga = [[0] for _ in range(N)]
  
  for h in range(N):
    for i in range(len(Nodos) - 1):
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h], T)
      Hormiga[h].append(Nuevo_Nodo)
      
    T = Incrementa_Feromona(problem, T, Hormiga[h])
    
    T = Evaporar_Feromonas(T)
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N):
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia = distancia_actual
    
def Add_Nodo(problem, H, T):
  Nodos = list(problem.get_nodes())
  return random.choice(list(set(range(1,len(Nodos))) - set(H)))

def Incrementa_Feromona(problem, T, H):
  for i in range(len(H) - 1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T
    
def Evaporar_Feromonas(T):
  T = [[max(T[i][j] -0.3, 1) for i in range(len(Nodos))] for j in range(len(Nodos))]
  return T
  
sol = hormigas(problem,100)